In [1]:
import cv2
import numpy as np
import glob
import collections

In [2]:
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [3]:
def object_detection(img):
    img = cv2.resize(img, None, fx=0.3, fy=0.3)
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.6:                          # setting the confidence level
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    M_labels = []
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            M_label = str(classes[class_ids[i]])
            M_labels.append(M_label)
            label = str(classes[class_ids[i]])+ " " +str(confidences[i])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 2, color, 3)
    #print(M_labels)            
    
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()          
    return (M_labels)

In [4]:
images = []

In [7]:
for img in glob.glob("pic/*.jpg"):
    #print (i)
    # Loading image
    n= cv2.imread(img)
    object_list = object_detection(n)
    #print (object_list)
    images.append(n) 
    counter=collections.Counter(object_list)
    print (object_list, counter['person']) 

['person'] 1
['person', 'sofa', 'banana'] 1
['tvmonitor'] 0
['person', 'person', 'teddy bear'] 2
['person'] 1
['person', 'person', 'person'] 3
['chair'] 0
['surfboard'] 0
['dog', 'dog', 'dog', 'person', 'person'] 2
[] 0
[] 0
[] 0
['car', 'car'] 0
['person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person'] 14
